In [ ]:
import base64
import json
import os
import re
import requests
import shutil
import traceback
import time

from openai import OpenAI

In [ ]:
def request_to_df(request_dict):
    df.dropna(inplace = True)

    # Dropping unused columns
    df.drop(['invoice_no', 'customer_id'], axis = 1, inplace = True)

    # Feature engineering
    df['total_purchase'] = df['price'] * df['quantity']  # Target variable

    # Dropping rows where 'total_purchase' exceeds 450
    df = df[df['total_purchase'] <= 450]
    df.reset_index(drop = True, inplace = True)

    # Duplicating the DataFrame 3 times
    df= pd.concat([df] * 3, ignore_index = True)

    # Convert invoice_date to datetime and extract useful features
    df['invoice_date'] = pd.to_datetime(df['invoice_date'], errors = 'coerce')

    # Encode categorical variables
    categorical_cols = ['gender', 'category', 'payment_method', 'shopping_mall']
    for col in categorical_cols:
        df[f"{col}_code"] = df[col].astype('category').cat.codes

    return df

In [ ]:
OPEN_AI_API_KEY = ""


def ask_chat_gpt(client, prompt):

    prompt = f"""Here is user prompt related to shopping preferences of people:

    {prompt}

    Please, convert it to nomalized format, that can be used in automazied processing. Convert prompt to JSON that containt following fields:
        - age:            integer
        - gender:         string, can be only 'Male' or 'Female'
        - category:       string, can be only 'Toys', 'Souvenir', 'Food & Beverage', 'Cosmetics' or 'Books'
        - payment_method: string, can be only 'Credit Card', 'Debit Card' or 'Cash'

    Try you best to fill all fields. If it is impossivle, use 'Unknown' as default value for string fields, and 30 - for age.

    Return only JSON with normalized data in plain string format."""

    messages = [
        {"role": "system", "content": "Ти - універсальний асистент, що добре розбирається в людях."},
    ]

    messages.append({
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
        ]
    })

    response = client.chat.completions.create(
        model      = "gpt-4o-mini", # "gpt-4-turbo",
        messages   = messages,
        max_tokens = 1300
    )

    message = response.choices[0].message.content

    return message

In [ ]:
ai_client = OpenAI(api_key = OPEN_AI_API_KEY)

while True:

    prompt = input()
    if prompt == "-":
        break

    responce = ask_chat_gpt(ai_client, prompt)

    start_index = responce.find('{')
    end_index   = responce.rfind('}')
    json_text   = responce[start_index:end_index + 1]

    request_dict = json.loads(json_text)
    request_df   = request_to_df(request_dict)

    print(json.loads(json_text))

-
